In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import geopandas as gpd
import folium
import branca.colormap as cm

# 1. 생활권 도출   

소생활권 기준인 2~3만명 기준으로 생활권을 도출한다.해볼 예정. 이 때, 읍면동을 기초 단위로 삼으며, 한 읍면동 당 3만명이 넘을 경우 하나의 읍면동을 하나의 생활권으로 묶는다.

In [42]:
df_human = pd.read_csv('data/human.csv', encoding = 'cp949')

In [43]:
df_human.head()

,행정기관코드,기준연월,시도명,시군구명,읍면동명,계,남자,여자,0세남자,1세남자,...,101세여자,102세여자,103세여자,104세여자,105세여자,106세여자,107세여자,108세여자,109세여자,110세이상 여자
0,1111051500,2025-03-31,서울특별시,종로구,청운효자동,10986,5007,5979,21,23,...,0,0,0,1,0,0,0,0,0,0
1,1111053000,2025-03-31,서울특별시,종로구,사직동,8937,3940,4997,17,8,...,2,1,0,0,0,0,0,0,0,0
2,1111054000,2025-03-31,서울특별시,종로구,삼청동,2189,1049,1140,5,2,...,0,0,0,0,0,0,0,0,0,0
3,1111055000,2025-03-31,서울특별시,종로구,부암동,9016,4229,4787,14,13,...,0,1,0,1,0,0,0,0,0,0
4,1111056000,2025-03-31,서울특별시,종로구,평창동,17233,7998,9235,37,38,...,2,1,2,0,1,0,0,0,0,0


In [44]:
df_human.describe()

,행정기관코드,계,남자,여자,0세남자,1세남자,2세남자,3세남자,4세남자,5세남자,...,101세여자,102세여자,103세여자,104세여자,105세여자,106세여자,107세여자,108세여자,109세여자,110세이상 여자
count,3.614000e+03,3614.000000,3614.000000,3614.000000,3614.000000,3614.000000,3614.000000,3614.000000,3614.000000,3614.000000,...,3614.000000,3614.000000,3614.000000,3614.000000,3614.000000,3614.000000,3614.000000,3614.000000,3614.000000,3614.000000
mean,3.832114e+09,14162.516879,7050.220808,7112.296071,33.835362,32.661040,35.513558,37.558661,39.166851,42.696735,...,0.464029,0.358329,0.208910,0.112064,0.065578,0.042335,0.024626,0.021859,0.015495,0.045102
std,1.261398e+09,13213.412572,6565.301239,6667.309112,46.512700,45.297711,50.077849,53.095971,55.446956,60.244795,...,0.751937,0.653074,0.477351,0.342414,0.255283,0.206806,0.156781,0.149982,0.123529,0.221741
min,1.111052e+09,93.000000,43.000000,50.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.826054e+09,3232.000000,1651.250000,1573.250000,3.000000,3.000000,3.000000,3.000000,3.000000,4.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.311134e+09,10522.000000,5292.000000,5208.000000,17.000000,16.500000,18.000000,19.000000,20.000000,22.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,4.773032e+09,21250.250000,10523.500000,10713.750000,48.000000,45.000000,48.750000,51.000000,53.000000,56.000000,...,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,5.280042e+09,116558.000000,57625.000000,58933.000000,513.000000,482.000000,569.000000,608.000000,635.000000,638.000000,...,6.000000,5.000000,4.000000,3.000000,2.000000,2.000000,2.000000,2.000000,1.000000,4.000000


## 1-1. 필요 컬럼만 남기고 지우기   
일단 나이도 3개 - total, 어린이, 노인 남기고 지움

In [45]:
# 어린이 컬럼 생성 (0~19세 남녀)
child_columns = [f"{age}세남자" for age in range(0, 20)] + [f"{age}세여자" for age in range(0, 20)]
df_human['어린이'] = df_human[child_columns].sum(axis=1)

# 노인 컬럼 생성 (65세 이상 남녀)
senior_male_columns = [f"{age}세남자" for age in range(65, 110)] + ["110세이상 남자"]
senior_female_columns = [f"{age}세여자" for age in range(65, 110)] + ["110세이상 여자"]
senior_columns = senior_male_columns + senior_female_columns
df_human['노인'] = df_human[senior_columns].sum(axis=1)


In [46]:
df_human = df_human[[
    '행정기관코드', '시도명', '시군구명', '읍면동명',
    '계', '어린이', '노인'
]]

In [47]:
df_human.head()

,행정기관코드,시도명,시군구명,읍면동명,계,어린이,노인
0,1111051500,서울특별시,종로구,청운효자동,10986,1554,2250
1,1111053000,서울특별시,종로구,사직동,8937,1020,1947
2,1111054000,서울특별시,종로구,삼청동,2189,234,627
3,1111055000,서울특별시,종로구,부암동,9016,1133,2006
4,1111056000,서울특별시,종로구,평창동,17233,2435,3868


In [48]:
df_human[df_human['읍면동명'].str.endswith('1동', na=False)] #'제N동', 'N동' 삭제 필요.

,행정기관코드,시도명,시군구명,읍면동명,계,어린이,노인
12,1111067000,서울특별시,종로구,창신제1동,4488,253,1314
15,1111070000,서울특별시,종로구,숭인제1동,5738,490,1363
36,1117056000,서울특별시,용산구,원효로제1동,17085,1979,2180
41,1117063000,서울특별시,용산구,이촌제1동,24954,4665,4637
43,1117065000,서울특별시,용산구,이태원제1동,6007,428,1448
...,...,...,...,...,...,...,...
3392,5211354000,전북특별자치도,전주시 덕진구,인후1동,17023,3008,3854
3398,5211361100,전북특별자치도,전주시 덕진구,우아1동,11758,1767,2451
3401,5211364100,전북특별자치도,전주시 덕진구,송천1동,64248,16285,6034
3428,5213070100,전북특별자치도,군산시,나운1동,12881,1509,3290


In [49]:
df_human.head()

,행정기관코드,시도명,시군구명,읍면동명,계,어린이,노인
0,1111051500,서울특별시,종로구,청운효자동,10986,1554,2250
1,1111053000,서울특별시,종로구,사직동,8937,1020,1947
2,1111054000,서울특별시,종로구,삼청동,2189,234,627
3,1111055000,서울특별시,종로구,부암동,9016,1133,2006
4,1111056000,서울특별시,종로구,평창동,17233,2435,3868


In [50]:
df_human.isna().sum()

행정기관코드    0
시도명       0
시군구명      5
읍면동명      0
계         0
어린이       0
노인        0
dtype: int64

In [51]:
missing_rows = df_human[df_human['시군구명'].isna()]
print(missing_rows)


          행정기관코드      시도명 시군구명 읍면동명      계   어린이    노인
1188  3611051500  세종특별자치시  NaN  새롬동  26328  7799  1829
1196  3611055500  세종특별자치시  NaN  소담동  21533  5762  1685
1197  3611055600  세종특별자치시  NaN  반곡동  28559  7495  1451
1199  3611057000  세종특별자치시  NaN  대평동  11140  2559  1013
1200  3611058000  세종특별자치시  NaN  다정동  28173  7957  2036


In [52]:
df_human.loc[df_human['시군구명'].isna(), '시군구명'] = '세종특별자치시'

In [53]:
df_human.isna().sum()
# 결측치 처리

행정기관코드    0
시도명       0
시군구명      0
읍면동명      0
계         0
어린이       0
노인        0
dtype: int64

## 1-2. 인구 수 별로 생활권 추출   

일단 근접하면서 + total 인구 수 비율 2~3만명 맞춰서 만들어보자

In [54]:
df = df_human.sort_values(by='행정기관코드')

results = []
zone_id = 1
temp_group = []
temp_population = 0
prev_code = None

# 생활권 단위로 묶어서 하나의 딕셔너리로 반환
def build_living_zone(cluster_rows, zone_id):
    cities = [str(row['시도명']) for row in cluster_rows]  # 중복 포함
    districts = [str(row['시군구명']) for row in cluster_rows]  # 중복 포함
    towns = [str(row['읍면동명']) for row in cluster_rows]  # 중복 포함
    codes = [str(row['행정기관코드']) for row in cluster_rows]  # 중복 포함

    total_population = sum(row['계'] for row in cluster_rows)
    total_children = sum(row['어린이'] for row in cluster_rows)
    total_seniors = sum(row['노인'] for row in cluster_rows)

    return {
        'ID': f"{zone_id:03}",
        '시도': ';'.join(cities),
        '시군구': ';'.join(districts),
        '읍면동': ';'.join(towns),
        '행정기관코드': ';'.join(codes),
        '총인구수': total_population,
        '어린이': total_children,
        '노인': total_seniors
    }

# 클러스터링 로직
for _, row in df.iterrows():
    row_dict = row.to_dict()
    code = str(int(row_dict['행정기관코드']))
    population = row_dict['계']

    if population >= 30000:
        results.append(build_living_zone([row_dict], zone_id))
        zone_id += 1
        continue

    if not temp_group:
        temp_group.append(row_dict)
        temp_population += population
        prev_code = code
        continue

    if code[:6] == prev_code[:6] and temp_population + population <= 30000:
        temp_group.append(row_dict)
        temp_population += population
        prev_code = code
    else:
        results.append(build_living_zone(temp_group, zone_id))
        zone_id += 1
        temp_group = [row_dict]
        temp_population = population
        prev_code = code

# 마지막 그룹 처리
if temp_group:
    results.append(build_living_zone(temp_group, zone_id))


# 최종 결과 DataFrame
df_living_zone = pd.DataFrame(results)

In [55]:
df_living_zone

,ID,시도,시군구,읍면동,행정기관코드,총인구수,어린이,노인
0,001,서울특별시;서울특별시;서울특별시,종로구;종로구;종로구,청운효자동;사직동;삼청동,1111051500;1111053000;1111054000,22112,2808,4824
1,002,서울특별시;서울특별시,종로구;종로구,부암동;평창동,1111055000;1111056000,26249,3568,5874
2,003,서울특별시;서울특별시,종로구;종로구,무악동;교남동,1111057000;1111058000,17341,2810,3318
3,004,서울특별시;서울특별시;서울특별시;서울특별시,종로구;종로구;종로구;종로구,가회동;종로1.2.3.4가동;종로5.6가동;이화동,1111060000;1111061500;1111063000;1111064000,22735,1624,5234
4,005,서울특별시;서울특별시;서울특별시,종로구;종로구;종로구,혜화동;창신제1동;창신제2동,1111065000;1111067000;1111068000,27638,2394,6065
...,...,...,...,...,...,...,...,...
2122,2123,전북특별자치도;전북특별자치도;전북특별자치도;전북특별자치도;전북특별자치도;전북특별자치...,고창군;고창군;고창군;고창군;고창군;고창군;고창군;고창군;고창군,고수면;아산면;무장면;공음면;상하면;해리면;성송면;대산면;심원면,5279031000;5279032000;5279033000;5279034000;52...,21112,1296,10488
2123,2124,전북특별자치도;전북특별자치도;전북특별자치도;전북특별자치도,고창군;고창군;고창군;고창군,흥덕면;성내면;신림면;부안면,5279040000;5279041000;5279042000;5279043000,9269,634,4607
2124,2125,전북특별자치도,부안군,부안읍,5280025000,20010,3386,5191
2125,2126,전북특별자치도;전북특별자치도;전북특별자치도;전북특별자치도;전북특별자치도;전북특별자치...,부안군;부안군;부안군;부안군;부안군;부안군;부안군;부안군;부안군,주산면;동진면;행안면;계화면;보안면;변산면;진서면;백산면;상서면,5280031000;5280032000;5280033000;5280034000;52...,21942,1251,10528


In [56]:
mixed_city = df_living_zone[df_living_zone['시도'].str.contains(';')]
mixed_district = df_living_zone[df_living_zone['시군구'].str.contains(';')]
print(f"혼합 도시 수: {len(mixed_city)}")
print(f"혼합 구 수: {len(mixed_district)}")

혼합 도시 수: 577
혼합 구 수: 577


### 생활권 편입   
생활권에 대한 병원 인프라를 조사하던 중 8곳의 생활권이 병원이 존재하지 않는 것을 알게 되었습니다. 

[주거지역을 생활권으로 인식한 케이스 2곳]
서울 둔춘제1동 : 아파트단지 모음, 인구수 29600
경기 은행1동 : 적당한 빌라촌. 인구수 9389
*주거지역=생활권으로 보기 애매함. 인프라가 X. 

[산지를 생활권으로 인식한 케이스 4곳]
부산 금성동, 경기 소요동, 경기 상패동, 경기 감북동;춘궁동
*다른 지역들 합치고 남은 애들인듯..

[멀쩡함 2곳]
충북 청주 봉명1동 : 시내임. 근데 병원이 길건너 있어서 0개로잡힘.
경남 밀양 교동 : 생활권은 맞는데 극소생활권. 옆동에 병원있음.
*여긴 그냥 정상적인 생활권인데 넓고, 병원 다 다른곳에 있는 case. 문제 없고 냅뒀다가 우선지역으로 넣을 예정  

따라서 주거지역=생활권, 산지=생활권 으로 인식한 6곳을 다른 생활권에 편입하기로 했습니다.
편입 조건은 제1동,제2동 처럼 가장 유사한 곳에 편입시키는 것을 1순위, 인접한 생활권 중 인구가 가장 낮은 곳에 편입시키는 것을 2순위로 잡아 다음과 같이 정했습니다.   

청룡노포동-금성동   
둔춘제2동-둔춘제1동 (여긴 사람수 낮은데랑 편입X 가장 유사한곳에 편입시킴, 동이나 시도반영)   
은행2동-은행1동(여긴 사람수 낮은데랑 편입X 가장 유사한곳에 편입시킴, 동이나 시도반영)   
상패동;1410-소요동 (상패동+소요동 합친담에 1410에 편입)   
소요동;1410-상패동   
초이동-감북동;춘궁동

In [62]:
df_before = df_living_zone.copy()
# 병합 리스트: (source_id, target_id) 형식
merge_list = [('381', '382'), ('488', '484'), ('985', '986'),('1120', '1410'), ('1121', '1410'), ('1266', '1269')]


In [ ]:
for from_id, to_id in merge_list:
    if from_id not in df_living_zone['ID'].values:
        continue  # ID가 없으면 skip

    row_from = df_living_zone[df_living_zone['ID'] == from_id].iloc[0]
    row_to = df_living_zone[df_living_zone['ID'] == to_id].iloc[0]

    # 문자열 필드 병합
    for col in ['시도', '시군구', '읍면동', '행정기관코드']:
        merged = f"{row_to[col]};{row_from[col]}"
        df_living_zone.loc[df_living_zone['ID'] == to_id, col] = merged

    # 수치 필드 병합
    for col in ['총인구수', '어린이', '노인']:
        df_living_zone.loc[df_living_zone['ID'] == to_id, col] += row_from[col]

# 병합된 from_id 모두 삭제
from_ids = [pair[0] for pair in merge_list]
df_living_zone = df_living_zone[~df_living_zone['ID'].isin(from_ids)].reset_index(drop=True)

병합 완료: 381 -> 382
병합된 행: ID               381
시도             서울특별시
시군구              강동구
읍면동            둔촌제1동
행정기관코드    1174069000
총인구수           29600
어린이             6167
노인              3932
Name: 380, dtype: object
병합 완료: 488 -> 484
병합된 행: ID               488
시도             부산광역시
시군구              금정구
읍면동              금성동
행정기관코드    2641070000
총인구수             903
어린이              126
노인               311
Name: 487, dtype: object
병합 완료: 985 -> 986
병합된 행: ID               985
시도               경기도
시군구          성남시 중원구
읍면동             은행1동
행정기관코드    4113355000
총인구수            9389
어린이             1151
노인              1830
Name: 984, dtype: object
병합 완료: 1120 -> 1410
병합된 행: ID              1120
시도               경기도
시군구             동두천시
읍면동              소요동
행정기관코드    4125058000
총인구수            8317
어린이              775
노인              2756
Name: 1119, dtype: object
병합 완료: 1121 -> 1410
병합된 행: ID              1121
시도               경기도
시군구             동두천시
읍면동              상패동
행정기관코드    4125

In [66]:

for from_id, to_id in merge_list:
    print(f"\n✅ 병합 확인: {from_id} ➝ {to_id}")
    print(df_living_zone[df_living_zone['ID'] == to_id][['ID', '읍면동', '행정기관코드', '총인구수', '어린이', '노인']].to_string(index=False))


✅ 병합 확인: 381 ➝ 382
 ID         읍면동                행정기관코드  총인구수  어린이   노인
382 둔촌제2동;둔촌제1동 1174070000;1174069000 54629 9535 8968

✅ 병합 확인: 488 ➝ 484
 ID       읍면동                행정기관코드  총인구수  어린이   노인
484 청룡노포동;금성동 2641066500;2641070000 10020 1070 3442

✅ 병합 확인: 985 ➝ 986
 ID       읍면동                행정기관코드  총인구수  어린이   노인
986 은행2동;은행1동 4113356000;4113355000 30753 3192 6954

✅ 병합 확인: 1120 ➝ 1410
  ID                                    읍면동                                                                                                        행정기관코드  총인구수  어린이    노인
1410 군남면;청산면;백학면;미산면;왕징면;신서면;중면;장남면;소요동;상패동 4180031000;4180032000;4180033000;4180034000;4180035000;4180036000;4180037000;4180038000;4125058000;4125060000 28154 2054 11051

✅ 병합 확인: 1121 ➝ 1410
  ID                                    읍면동                                                                                                        행정기관코드  총인구수  어린이    노인
1410 군남면;청산면;백학면;미산면;왕징면;신서면;중면;장남면;소요동;상패동 4180031000;4180032000;4

In [67]:
df_living_zone.to_csv('최종생활권.csv', index=False, encoding='utf-8-sig')

In [68]:
gdf = gpd.read_file("data\HangJeongDong_ver20250401.geojson")
print(gdf.columns)
print(gdf.head(1))

<>:1: SyntaxWarning: invalid escape sequence '\H'
<>:1: SyntaxWarning: invalid escape sequence '\H'
C:\Users\zappe\AppData\Local\Temp\ipykernel_57356\1151699018.py:1: SyntaxWarning: invalid escape sequence '\H'
  gdf = gpd.read_file("data\HangJeongDong_ver20250401.geojson")


Index(['adm_nm', 'adm_cd2', 'sgg', 'sido', 'sidonm', 'sggnm', 'adm_cd',
       'geometry'],
      dtype='object')
          adm_nm     adm_cd2    sgg sido sidonm sggnm    adm_cd  \
0  서울특별시 종로구 사직동  1111053000  11110   11  서울특별시   종로구  11010530   

                                            geometry  
0  MULTIPOLYGON (((126.97689 37.57565, 126.97703 ...  


In [69]:

geojson_path = "data/HangJeongDong_ver20250401.geojson"
gdf = gpd.read_file(geojson_path, ignore_geometry=True)
print(gdf.columns)

Index(['adm_nm', 'adm_cd2', 'sgg', 'sido', 'sidonm', 'sggnm', 'adm_cd'], dtype='object')


In [76]:
# 1. 데이터 불러오기
df_living = pd.read_csv("data/최종생활권.csv")
gdf = gpd.read_file("data/HangJeongDong_ver20250401.geojson")

# 2. 행정기관코드 분해 및 병합
df_exp = df_living.copy()
df_exp = df_exp.assign(행정기관코드=df_exp['행정기관코드'].astype(str).str.split(';')).explode('행정기관코드')
df_exp['행정기관코드'] = df_exp['행정기관코드'].str.strip()
df_exp['행정기관코드'] = df_exp['행정기관코드'].astype(str)

gdf['adm_cd2'] = gdf['adm_cd2'].astype(str)
gdf_merged = gdf.merge(df_exp, left_on='adm_cd2', right_on='행정기관코드', how='left')

# 3. 인구 수치는 ID 기준 groupby로 정확히 합산
pop_cols = ['총인구수', '어린이', '노인']
df_pop = gdf_merged.groupby('ID')[pop_cols].sum().reset_index()

# 4. geometry는 ID 기준으로 dissolve (폴리곤 병합)
gdf_dissolved = gdf_merged.dissolve(by='ID', as_index=False)

# 5. 병합: geometry + 인구 통계 데이터
gdf_living_zone = gdf_dissolved.merge(df_pop, on='ID', how='left')

for col in ['총인구수', '어린이', '노인']:
    col_x = f'{col}_x'
    col_y = f'{col}_y'
    if col_x in gdf_living_zone.columns and col_y in gdf_living_zone.columns:
        gdf_living_zone[col] = gdf_living_zone[col_x].combine_first(gdf_living_zone[col_y])
        gdf_living_zone.drop(columns=[col_x, col_y], inplace=True)

# 6. 색상 맵 정의
colormap = cm.linear.YlGnBu_09.scale(
    gdf_living_zone['총인구수'].min(),
    gdf_living_zone['총인구수'].max()
)
colormap.caption = '생활권 총인구수'

# 7. Folium 지도 시각화
m = folium.Map(location=[36.5, 127.8], zoom_start=7)

folium.GeoJson(
    gdf_living_zone,
    name="생활권 단위",
    tooltip=folium.GeoJsonTooltip(
        fields=['ID', '읍면동', '총인구수', '어린이', '노인'],
        aliases=['ID', '읍면동', '총인구수', '어린이', '노인'],
        localize=True
    ),
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['총인구수']) if feature['properties']['총인구수'] else 'gray',
        'color': 'black',
        'weight': 0.3,
        'fillOpacity': 0.6,
    }
).add_to(m)

colormap.add_to(m)

# 8. 저장
m.save("only생활권.html")
gdf_living_zone.to_file("only생활권.geojson", driver="GeoJSON")
